In [1]:
import numpy as np
import pandas as pd

In [2]:
temp_df = pd.read_csv('IMDB Dataset.csv')

In [3]:
df = temp_df.iloc[:10000]

In [4]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
df.drop_duplicates(inplace=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_9588\3006716147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [6]:
import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text

In [7]:
df['review'] = df['review'].apply(remove_tags)

C:\Users\HP\AppData\Local\Temp\ipykernel_9588\2336150696.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(remove_tags)


In [8]:
df['review'] = df['review'].apply(lambda x:x.lower())

C:\Users\HP\AppData\Local\Temp\ipykernel_9588\740760900.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x:x.lower())


In [9]:
from nltk.corpus import stopwords

sw_list = stopwords.words('english')

df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

C:\Users\HP\AppData\Local\Temp\ipykernel_9588\2826946130.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))


In [10]:
df['review']

0       one reviewers mentioned watching 1 oz episode ...
1       wonderful little production. filming technique...
2       thought wonderful way spend time hot summer we...
3       basically there's family little boy (jake) thi...
4       petter mattei's "love time money" visually stu...
                              ...                        
9995    fun, entertaining movie wwii german spy (julie...
9996    give break. anyone say "good hockey movie"? kn...
9997    movie bad movie. watching endless series bad h...
9998    movie probably made entertain middle school, e...
9999    smashing film film-making. shows intense stran...
Name: review, Length: 9983, dtype: object

In [11]:
import gensim

In [12]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [13]:
story = []
for doc in df['review']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))

In [14]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [15]:
model.build_vocab(story)

In [16]:
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

(5849911, 6186875)

In [17]:
len(model.wv.index_to_key)

31845

In [18]:
def document_vector(doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [19]:
document_vector(df['review'].values[0])

array([-0.00763949,  0.38470912,  0.17476937,  0.02096314, -0.03312165,
       -0.51519436,  0.11321089,  0.889526  , -0.22312115, -0.2261461 ,
       -0.28570607, -0.4484552 ,  0.00343308,  0.22603002,  0.14373033,
       -0.23709859,  0.08480971, -0.5790368 , -0.02049911, -0.6638871 ,
        0.07815514,  0.04833188,  0.2106966 , -0.35910898, -0.17154582,
       -0.18248951, -0.18263924, -0.20362538, -0.34909266, -0.04813133,
        0.31230488,  0.01781886, -0.00961612, -0.18493481, -0.39134988,
        0.3677735 ,  0.0372747 , -0.2756002 , -0.18182857, -0.7501857 ,
        0.0827388 , -0.2800884 , -0.12187809, -0.03029208,  0.44944686,
       -0.13985145, -0.39532852, -0.21134914,  0.07541313,  0.28488526,
        0.16406837, -0.22404076, -0.39509097, -0.2341627 , -0.42382818,
        0.08190946,  0.42633513,  0.19559886, -0.32121634,  0.0272052 ,
       -0.08791725,  0.23063663, -0.16547583, -0.04622761, -0.43384236,
        0.41427806, -0.02832825,  0.14720033, -0.50083053,  0.34

In [20]:
from tqdm import tqdm

In [21]:
X = []
for doc in tqdm(df['review'].values):
    X.append(document_vector(doc))

100%|██████████████████████████████████████████████████████████████████████████████| 9983/9983 [24:21<00:00,  6.83it/s]


In [22]:
X = np.array(X)

In [23]:
X[0]

array([-0.00763949,  0.38470912,  0.17476937,  0.02096314, -0.03312165,
       -0.51519436,  0.11321089,  0.889526  , -0.22312115, -0.2261461 ,
       -0.28570607, -0.4484552 ,  0.00343308,  0.22603002,  0.14373033,
       -0.23709859,  0.08480971, -0.5790368 , -0.02049911, -0.6638871 ,
        0.07815514,  0.04833188,  0.2106966 , -0.35910898, -0.17154582,
       -0.18248951, -0.18263924, -0.20362538, -0.34909266, -0.04813133,
        0.31230488,  0.01781886, -0.00961612, -0.18493481, -0.39134988,
        0.3677735 ,  0.0372747 , -0.2756002 , -0.18182857, -0.7501857 ,
        0.0827388 , -0.2800884 , -0.12187809, -0.03029208,  0.44944686,
       -0.13985145, -0.39532852, -0.21134914,  0.07541313,  0.28488526,
        0.16406837, -0.22404076, -0.39509097, -0.2341627 , -0.42382818,
        0.08190946,  0.42633513,  0.19559886, -0.32121634,  0.0272052 ,
       -0.08791725,  0.23063663, -0.16547583, -0.04622761, -0.43384236,
        0.41427806, -0.02832825,  0.14720033, -0.50083053,  0.34

In [24]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

y = encoder.fit_transform(df['sentiment'])

In [25]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [26]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [28]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7816725087631448